# Validation

This notebook contains the steps followed to build the validated networks starting from the original retweet network

In [1]:
import numpy as np
import networkx as nx
from NEMtropy import DirectedGraph
from NEMtropy.network_functions import build_adjacency_from_edgelist
from tqdm import tqdm
import matplotlib.pyplot as plt
from validation import Graph_validation

import graph_tool.all as gt

### Here we are going to estimate the parameter of the model CReMa.

Let's load all the relevant information regarding the original graph.

In [2]:
# Upload the graph
G = gt.load_graph("Left.gt")

# Take the property "weight" associate to each link
weights = G.edge_properties["weight"]

# Create the weighted adjacency matrix
adj_weigh = gt.adjacency(G, weight=weights)


# get the property map for all the IDs for each node
id_map = G.vertex_properties["id"]

# initialize an empty list to save the IDs
ordered_ids = []

# Loop through all the vertices of the graph and save their IDs in the list.
for v in G.vertices():
    ordered_ids.append(id_map[v])

FileNotFoundError: [Errno 2] No such file or directory: 'Left.gt'

We prepare the metrics that will be inserted into the CReMa to estimate the parameters.

In [ ]:
# Lets store the binary structure and strength sequence
adj_bin = adj_weigh.astype(bool).astype(int)

out_strength =  np.array(adj_weigh.sum(axis=1)).flatten() 
in_strength =  np.array(adj_weigh.sum(axis=0)).flatten() 

# We can initialiase our graph instance with the strength sequence
graph_weighted = DirectedGraph(strength_sequence=np.concatenate([out_strength, in_strength]))

We use the internal solver to maximize the likelihood and estimate the parameters.

In [2]:
graph_weighted.solve_tool(model="crema",
                          method="quasinewton",
                          initial_guess=initial_values,
                          adjacency=adj_bin,
                          verbose=False,
                          max_steps=1)

In [13]:
b_in = graph_weighted.b_in
b_out = graph_weighted.b_out

Once the parameters are estimated, we can proceed with the validation of the links using the 'Graph_validation' function.

In [ ]:
validated_edges = Graph_validation(adj_weigh,ordered_ids,b_in,b_out)

As output, we have the list of all edges that have been determined as significant according to our model with the associated weights.

Given the edge list, one can proceed to create a new graph that will contain only the valid links.

In [ ]:
# Create a directed graph
graph = gt.Graph(directed=True)

# Add vertices
vertex_ids = set(edge[0] for edge in validated_edges) | set(edge[1] for edge in validated_edges)
vertex_map = {v: graph.add_vertex() for v in vertex_ids}
edges = [(vertex_map[source], vertex_map[target]) for source, target,weight in  validated_edges]


graph.add_edge_list(edges)

# Add weights as edge property
weights = graph.new_edge_property("double")
for edge, weight in zip(graph.edges(), validated_edges):
    weights[edge] = weight[2]

graph.edge_properties["weight"] = weights

inverted_vmap = {j:i for i,j in vertex_map.items()}
vertex_ids = graph.new_vertex_property("string")
for i in graph.vertices():
    vertex_ids[i] = inverted_vmap[i]

graph.vertex_properties["id"] = vertex_ids
#Need to define a the path where you want to save the graph.
graph.save(path_to_save + "Left.gt")
